In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import acquire_zillow
from acquire_zillow import get_connection

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

import scipy.stats as stats
from scipy.stats import pearsonr

import graphviz
from graphviz import Graph

import env

In [2]:
from acquire_zillow import get_properties_2016
from acquire_zillow import get_properties_2017
from acquire_zillow import get_zillow_csv


In [3]:
get_zillow_csv()

In [56]:
df = pd.read_csv('zillow_properties.csv')

In [57]:
df.shape

(167854, 68)

In [58]:
df.tail().T

,167849,167850,167851,167852,167853
id,2864704,673515,2968375,1843709,1187175
parcelid,10833991,11000655,17239384,12773139,12826780
airconditioningtypeid,1,NaN,NaN,1,NaN
architecturalstyletypeid,NaN,NaN,NaN,NaN,NaN
basementsqft,NaN,NaN,NaN,NaN,NaN
bathroomcnt,3,2,2,1,2
bedroomcnt,3,2,4,3,3
buildingclasstypeid,NaN,NaN,NaN,NaN,NaN
buildingqualitytypeid,8,6,NaN,4,6
calculatedbathnbr,3,2,2,1,2


In [59]:
def reindex_zillow(df):
    return df.reindex_axis(['parcelid',
 'logerror',
 'bedroomcnt',
 'bathroomcnt',
 'calculatedfinishedsquarefeet',
 'transactiondate',
 'yearbuilt',
 'fips',
 'latitude',
 'longitude',
 'airconditioningtypeid',
 'airconditioningdesc',
 'architecturalstyletypeid',
 'architecturalstyledesc',
 'basementsqft',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'buildingclassdesc',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'heatingorsystemdesc',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertylandusedesc',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'storydesc',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'typeconstructiondesc',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'numberofstories',
 'fireplaceflag',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxamount',
 'taxdelinquencyflag',
 'taxdelinquencyyear',
 'censustractandblock'], axis=1) 

In [60]:
df['propertylandusedesc'].value_counts()

Single Family Residential                     113078
Condominium                                    42157
Duplex (2 Units, Any Combination)               4397
Planned Unit Development                        4281
Quadruplex (4 Units, Any Combination)           1608
Triplex (3 Units, Any Combination)              1168
Cluster Home                                     691
Mobile Home                                      158
Manufactured, Modular, Prefabricated Homes       105
Residential General                              104
Cooperative                                       57
Commercial/Office/Residential Mixed Used          32
Townhouse                                         17
Store/Office (Mixed Use)                           1
Name: propertylandusedesc, dtype: int64

In [61]:
df = reindex_zillow(df)
df.sample(10).T

,143245,165971,78553,145299,112466,3275,36325,135194,33396,129068
parcelid,14670659,11417242,11250090,12013983,14382559,11078404,17083319,12864941,12933112,14754465
logerror,0.00772724,0.132119,-0.1696,-0.0229566,0.00520157,0.0583,0.6981,0.155178,0.008,0.0465317
bedroomcnt,2,3,4,2,4,4,3,2,3,1
bathroomcnt,2,2,2,2,2,3,2,2,3,1
calculatedfinishedsquarefeet,1031,1404,1839,1065,1867,2966,1465,1118,1842,685
transactiondate,2017-06-30,2017-09-12,2016-09-22,2017-07-07,2017-03-31,2016-01-19,2016-05-18,2017-06-09,2016-05-09,2017-05-23
yearbuilt,1989,2004,1978,1966,1975,1976,1973,1972,1954,1984
fips,6059,6037,6037,6037,6059,6037,6111,6037,6037,6059
latitude,3.35977e+07,3.39328e+07,3.46129e+07,3.41173e+07,3.36003e+07,3.42477e+07,3.42781e+07,3.41097e+07,3.4073e+07,3.35704e+07
longitude,-1.1772e+08,-1.18355e+08,-1.17812e+08,-1.1834e+08,-1.17694e+08,-1.18623e+08,-1.1924e+08,-1.17737e+08,-1.17869e+08,-1.17709e+08


#### Include only single unit properties (e.g. no duplexes, no land/lot, ...) For some properties, you will need to use multiple fields to estimate whether it is a single unit property.

In [62]:
df.loc[(df['propertylandusedesc'] != 'Single Family Residential') &\
      (df['propertylandusedesc'] != 'Mobile Home') &\
      (df['propertylandusedesc'] != 'Manufactured, Modular, Prefabricated Homes')]

,parcelid,logerror,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,transactiondate,yearbuilt,fips,latitude,longitude,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
3,12643413,0.021800,2.0,2.0,839.0,2016-01-02,1987.0,6037.0,33755800.0,-118309000.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
7,17177301,0.163800,3.0,2.5,1763.0,2016-01-02,1994.0,6111.0,34207204.0,-119165589.0,...,2.0,NaN,107000.0,306000.0,2015.0,199000.0,3745.50,NaN,NaN,6.111003e+13
8,14739064,-0.003000,2.0,1.0,796.0,2016-01-02,1984.0,6059.0,33549600.0,-117678000.0,...,NaN,NaN,66834.0,210064.0,2015.0,143230.0,2172.88,NaN,NaN,6.059042e+13
9,14677559,0.084300,2.0,2.0,1260.0,2016-01-03,1977.0,6059.0,33612700.0,-117742000.0,...,1.0,NaN,109977.0,190960.0,2015.0,80983.0,1940.26,NaN,NaN,6.059063e+13
13,14665244,0.081600,2.0,2.0,1249.0,2016-01-03,2005.0,6059.0,33670900.0,-117844000.0,...,NaN,NaN,398747.0,601000.0,2015.0,202253.0,6876.70,NaN,NaN,6.059063e+13
16,17296797,0.044000,3.0,2.5,1558.0,2016-01-03,1979.0,6111.0,34176362.0,-118757591.0,...,2.0,NaN,168000.0,480000.0,2015.0,312000.0,6383.74,NaN,NaN,6.111007e+13
17,12524288,-0.041900,1.0,1.0,597.0,2016-01-03,1970.0,6037.0,33776400.0,-118123000.0,...,NaN,NaN,56233.0,70316.0,2015.0,14083.0,913.17,NaN,NaN,6.037575e+13
27,14646051,0.008000,2.0,2.5,1858.0,2016-01-03,1973.0,6059.0,33486586.0,-117722058.0,...,2.0,1.0,145291.0,825000.0,2015.0,679709.0,9553.56,NaN,NaN,6.059042e+13
28,10805063,0.001000,2.0,2.0,1045.0,2016-01-03,1966.0,6037.0,34167800.0,-118522000.0,...,NaN,NaN,135218.0,250245.0,2015.0,115027.0,3117.19,NaN,NaN,6.037140e+13
29,12524725,-0.053400,2.0,3.0,1329.0,2016-01-03,1983.0,6037.0,33780400.0,-118141000.0,...,NaN,NaN,117504.0,215097.0,2015.0,97593.0,2648.96,NaN,NaN,6.037577e+13


The following function will remove any listings that have 0 restrooms, 0 bedrooms, or a unincnt listing with 2 or more.  Additionally, it will remove all listings that are not either listed as Single Family Residential, Mobile Home, or Manufactured, Modular, Prefrabricated Homes in the propertylandusedesc column.

In [63]:
def drop_land_listings(df):
    df.drop(df[df['bedroomcnt'] == 0.0].index, inplace=True)
    df.drop(df[df['bathroomcnt'] == 0.0].index, inplace=True)
    df.drop(df[df['unitcnt'] >= 2.0].index, inplace=True)
    df.drop(df[(df['propertylandusedesc'] != 'Single Family Residential') &\
      (df['propertylandusedesc'] != 'Mobile Home') &\
      (df['propertylandusedesc'] != 'Manufactured, Modular, Prefabricated Homes')].index, inplace=True)
    return df

In [64]:
drop_land_listings(df)
df.shape

(112689, 67)

Creat a function that takes in a dataframe and a list of columns names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.
Use this function to appropriately transform any numeric columns that should not be treated as numbers



Looking at this list below, transaction date and propertycountlandusecode should be numerical instead of objects.
- Update: Propertycountylandusecode has multiple rows with mixed numeric and alpha values.  Transactiondate will be the only feature that will be converted to numeric.

In [73]:
df.select_dtypes('number').sample(10).T

,18182,112309,136973,95037,108058,4173,2454,93664,120733,16308
parcelid,1.400976e+07,1.259328e+07,1.109288e+07,1.387370e+07,1.136159e+07,1.715911e+07,1.111721e+07,1.433308e+07,1.088652e+07,1.280728e+07
logerror,2.000000e-03,-1.329859e-01,-9.001305e-03,1.503353e-02,9.675103e-03,2.000000e-03,1.880000e-02,1.011267e-01,2.945593e-01,3.440000e-02
bedroomcnt,3.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00
bathroomcnt,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,3.000000e+00,4.500000e+00,2.000000e+00,2.000000e+00
calculatedfinishedsquarefeet,1.858000e+03,1.172000e+03,2.793000e+03,1.937000e+03,1.814000e+03,1.350000e+03,3.189000e+03,4.382000e+03,1.620000e+03,1.272000e+03
transactiondate,2.016032e+07,2.017033e+07,2.017062e+07,2.017012e+07,2.017032e+07,2.016012e+07,2.016011e+07,2.017012e+07,2.017043e+07,2.016032e+07
yearbuilt,1.961000e+03,1.960000e+03,1.953000e+03,1.922000e+03,1.989000e+03,1.962000e+03,2.013000e+03,1.992000e+03,1.954000e+03,1.950000e+03
fips,6.059000e+03,6.037000e+03,6.037000e+03,6.059000e+03,6.037000e+03,6.111000e+03,6.037000e+03,6.059000e+03,6.037000e+03,6.037000e+03
latitude,3.377886e+07,3.384130e+07,3.423657e+07,3.387419e+07,3.445176e+07,3.421842e+07,3.446293e+07,3.374590e+07,3.418333e+07,3.396692e+07
longitude,-1.179630e+08,-1.182914e+08,-1.185207e+08,-1.179302e+08,-1.186601e+08,-1.191920e+08,-1.185647e+08,-1.177868e+08,-1.184180e+08,-1.180713e+08


In [65]:
df.select_dtypes('object')

,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertycountylandusecode,propertylandusedesc,propertyzoningdesc,storydesc,typeconstructiondesc,taxdelinquencyflag
0,2016-01-01,Central,NaN,NaN,Central,0100,Single Family Residential,LARS,NaN,NaN,NaN
1,2016-01-01,NaN,NaN,NaN,NaN,1,Single Family Residential,NaN,NaN,NaN,NaN
2,2016-01-01,Central,NaN,NaN,Central,0100,Single Family Residential,PSR6,NaN,NaN,NaN
4,2016-01-02,NaN,NaN,NaN,NaN,122,Single Family Residential,NaN,NaN,NaN,NaN
5,2016-01-02,Central,NaN,NaN,Central,0100,Single Family Residential,HBR3YY,NaN,NaN,NaN
6,2016-01-02,NaN,NaN,NaN,Floor/Wall,0100,Single Family Residential,CORH*,NaN,NaN,NaN
10,2016-01-03,NaN,NaN,NaN,Central,0100,Single Family Residential,LARE9,NaN,NaN,NaN
11,2016-01-03,Central,NaN,NaN,Central,0100,Single Family Residential,LAR1,NaN,NaN,NaN
12,2016-01-03,NaN,NaN,NaN,Floor/Wall,0100,Single Family Residential,BUR1*,NaN,NaN,NaN
14,2016-01-03,NaN,NaN,NaN,NaN,122,Single Family Residential,NaN,NaN,NaN,NaN


In [66]:
df['transactiondate'] = pd.to_numeric(df.transactiondate.str.replace('-', ''))
df.head()


,parcelid,logerror,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,transactiondate,yearbuilt,fips,latitude,longitude,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,3.0,2.0,1684.0,20160101,1959.0,6037.0,34280990.0,-118488536.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,4.0,3.5,2263.0,20160101,2014.0,6059.0,33668120.0,-117677556.0,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2.0,3.0,2217.0,20160101,1940.0,6037.0,34136312.0,-118175032.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
4,14432541,-0.0050,4.0,2.5,2283.0,20160102,1981.0,6059.0,33485643.0,-117700234.0,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
5,11509835,-0.2705,4.0,4.0,3067.0,20160102,1982.0,6037.0,33870089.0,-118402768.0,...,NaN,NaN,880650.0,2447951.0,2015.0,1567301.0,27126.57,NaN,NaN,6.037621e+13


In [69]:
cols_to_numeric = ['transactiondate']
def convert_date_object_to_int(df, list_of_cols):
    for col in list_of_cols:
        df[[col]] = df[[col]].astype(int)
    return df
    

In [71]:
convert_date_object_to_int(df, cols_to_numeric)
df.head()

,parcelid,logerror,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,transactiondate,yearbuilt,fips,latitude,longitude,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,3.0,2.0,1684.0,20160101,1959.0,6037.0,34280990.0,-118488536.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,4.0,3.5,2263.0,20160101,2014.0,6059.0,33668120.0,-117677556.0,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2.0,3.0,2217.0,20160101,1940.0,6037.0,34136312.0,-118175032.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
4,14432541,-0.0050,4.0,2.5,2283.0,20160102,1981.0,6059.0,33485643.0,-117700234.0,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
5,11509835,-0.2705,4.0,4.0,3067.0,20160102,1982.0,6037.0,33870089.0,-118402768.0,...,NaN,NaN,880650.0,2447951.0,2015.0,1567301.0,27126.57,NaN,NaN,6.037621e+13
